Task 1: Shunyu

• reviews with less than two sentences in the review body.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Stage two") \
    .getOrCreate()

In [ ]:
music_data = 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz'

In [ ]:
musics= spark.read.csv(music_data,header=True,sep='\t')

In [ ]:
import nltk
nltk.download('punkt')

In [0]:
from pyspark.sql.functions import *
from nltk.tokenize import sent_tokenize
def sent_token(s):
    sent_list = sent_tokenize(str(s))
    num = len(sent_list)
    return int(num)
slen = udf(lambda s: sent_token(s), IntegerType())
musics=musics.withColumn('sent_num',slen(musics.review_body).alias('slen'))
musics.filter(musics.sent_num >= 2)
musics.show()

Task 2: Keyi

• reviews from products with less than median number of reviews received
• reviews published by users with less than median number of reviews published

In [0]:
music0 = musics_first.groupBy('product_id').count()
music1 = music0.withColumn("rowNum", row_number().over(Window.orderBy("count")))#partitionBy give different ranks for different XXX
rows = music1.agg(collect_list("rowNum")).collect()
begin = rows[0][0][0]
end = rows[0][0][-1]
med = int((begin + end) /2)
print (med)
keys=music1.filter(music1.rowNum>med).agg(collect_list("product_id")).collect()
#print(keys[0][0])
musics_first.filter(musics.product_id.isin(keys[0][0])).show(10)

In [0]:
music0 = musics_first.groupBy('customer_id').count()
music1 = music0.withColumn("rowNum", row_number().over(Window.orderBy("count")))#partitionBy give different ranks for different XXX
rows = music1.agg(collect_list("rowNum")).collect()
begin = rows[0][0][0]
end = rows[0][0][-1]
med = int((begin + end) /2)
print (med)
keys=music1.filter(music1.rowNum>med).agg(collect_list("customer_id")).collect()
print(keys[0][0])
musics_first.filter(musics.customer_id.isin(keys[0][0])).show(10)

task 3: yuling

• top 10 users ranked by median number of sentences in the reviews they have pub- lished;
• top 10 products ranked by median number of sentences in the reviews they have received;

In [ ]:
##top 10 users ranked by median number of sentences in the reviews they have received

import pyspark.sql.functions
import numpy as np
from pyspark.sql.types import FloatType

def median(lists):
    try:
        median = np.median(lists)
        return float(median)
    except Exception:
        return None 

median_finder = udf(median,FloatType())
slen = udf(lambda s: len(sent_tokenize(s)), IntegerType())
sentences=musics_first.withColumn('sent_num',slen(musics['review_body']).alias('slen'))
sentences_list = sentences.groupBy(musics_first['customer_id']).agg(collect_list("sent_num").alias("lists"))
sentences_list1 = sentences_list.withColumn('median',median_finder('lists'))
top10users = sentences_list1.orderBy(desc("median"))
top10users.show(10)

In [ ]:
##top 10 products ranked by median number of sentences in the reviews they have received

product_list = musics_first.groupBy('product_id').agg(collect_list("sent_num").alias('lists_sentnum'))
product_list1 = product_list.withColumn('median',median_finder('lists_sentnum'))
top10products = product_list1.orderBy(desc('median'))
top10products.show(10)